In [16]:
import os
from datetime import datetime
from src.python.classes.DBData import DBData
from src.python.classes.CallsClass import CallsClass
from duckdb import query

In [15]:
# carregando edições para filtro em queries
edicoes = DBData().ScrapDB(
    query_or_list_editions=2,
    edicao='10 Milhas'
)
edicoes

Edições carregadas.


,globalEvent
0,10 Milhas
1,10 Milhas - serviços


In [18]:
# consultando e exportando xlsx dos formulários e participantes p/ paths respectivos
conn_query = DBData().ScrapDB(
    query_or_list_editions=1,
    edicao='Maratona do Rio 2025',
    data_compra_ini='2022-05-01',
    data_compra_fini='2026-05-31',
    limit_max_rows=1000000,
    total_rows_in_batches=2500
)


consulting ceps...
Total de registros carregados e exportados: 56604 ceps.

consulting participants...
Total de registros carregados e exportados: 57958 participants.
Forms e participants carregados e exportados.


In [20]:
# carregando xlsx de participantes em df
participants = CallsClass().load_and_parse_participants()

Arquivos encontrados:
 ['participants_results_10_Milhas.xlsx', 'participants_results_Desafio_da_Ponte.xlsx', 'participants_results_Maratona_do_Rio_2025.xlsx']
Arquivo de participantes carregado.


In [21]:
# carregando xlsx de formulários em df
forms = CallsClass().load_and_parse_forms()

Arquivos encontrados:
 ['forms_results_10_Milhas.xlsx', 'forms_results_Desafio_da_Ponte.xlsx', 'forms_results_Maratona_do_Rio_2025.xlsx']

Arquivo carregado. Parseando...
+------------------------------+------------------+
|                              | qtde (percent)   |
|------------------------------+------------------|
| total de ceps                | 56604 (100.00%)  |
| ceps com underline           | 728 (1.29%)      |
| ceps sem hifen               | 179 (0.32%)      |
| ceps ok                      | 56587 (99.97%)   |
| ceps com menos de 8 dígitos  | 4 (0.01%)        |
| ceps com mais de 8 dígitos   | 13 (0.02%)       |
| ceps com tratamento especial | 0 (0.00%)        |
+------------------------------+------------------+
Parseado com sucesso.


In [11]:
# Gets dos ceps no forms pela col 'parsed_ceps'. Do dict retornado, 'complete_api_df' contém os endereços.
gets = CallsClass().triforce(
    parsed_ceps_df=forms,
    timeout=5
)
addresses_df = gets.get('complete_api_df')

100%|██████████| 129/129 [13:10<00:00,  5.72s/it]

ceps logs
+------+-------+
|   ok |   nok |
|------+-------|
|  127 |     2 |
+------+-------+
+---------------------+-----------------+-----------------+
|   brasil api errors |   viacep errors |   apicep errors |
|---------------------+-----------------+-----------------|
|                   2 |               2 |               2 |
+---------------------+-----------------+-----------------+


100%|██████████| 129/129 [13:15<00:00,  6.17s/it]


In [12]:
# check
addresses_df.head()

,item_id,cep,state,city,neighborhood,street,service
0,482892,25520731,RJ,São João de Meriti,Vila Tiradentes,Rua Laura Arruda,open-cep
1,482893,25545190,RJ,São João de Meriti,Éden,Rua Irmãos Guinle,open-cep
2,482955,25520640,RJ,São João de Meriti,Centro,Rua da Matriz,open-cep
3,482956,25520640,RJ,São João de Meriti,Centro,Rua da Matriz,open-cep
4,482973,25555120,RJ,São João de Meriti,Jardim Meriti,Rua Otávio Mangabeira,open-cep


In [13]:
# gerando df para export
participants_and_addresses = query(
    query='''
    SELECT 
      participants.*
    , addresses_df.cep as cep
    , addresses_df.state as UF
    , addresses_df.city as cidade
    , addresses_df.neighborhood as bairro
    , addresses_df.street as logradouro
    FROM participants
    LEFT JOIN addresses_df
    ON participants.itemID = addresses_df.item_id
    '''   
).to_df()

In [14]:
# exportando df resultante do evento para xlsx

folder = os.path.join(os.getcwd(), 'src', 'datasets', 'results')
os.makedirs(folder, exist_ok=True)

edition = participants['edição'].unique()[0].replace(' ','_')
current_time = datetime.now().strftime("%Y%m%d_%H%M")

result_df_path = os.path.join(folder, f'{edition}_{current_time}.xlsx')
participants_and_addresses.to_excel(result_df_path, index=False)

print(f'Arquivo {edition}_{current_time}.xlsx exportado.\npath: {result_df_path}')

Arquivo 10_Milhas_20250820_1240.xlsx exportado.
path: c:\Users\vitor\Desktop\CepsAPIs\src\datasets\results\10_Milhas_20250820_1240.xlsx
